# Imports

In [1]:
import requests
import json
import pprint
from bs4 import BeautifulSoup as bs
import pandas as pd


# Implementation

In [2]:
url = "https://content.guardianapis.com/search"
content = ["covid"] #you can have several keywords
apiKey = "5ee2a23b-f46e-4986-a548-e4d9e5bd569e"
if 1: #if you specified author then switch to if 0
    authorSpecified = ""
else:
    authorSpecified = "Nina Lakhani" #change name
#chage range yyyy--mm-dd 
upperYear = 2022
lowerYear = 2000
fromDateMonth = "-03-11"
toDateMonth = "-03-12"


In [3]:
def generateYears(fromDY,toDY):
    for i in range(lowerYear,upperYear+1):
        fromDY.append(str(i)+fromDateMonth)
        toDY.append(str(i)+toDateMonth)

In [4]:
def loopYears(content,links):
    for i, j in zip(fromDateYear,toDateYear):
        parameters = {
            'q': content,
            'pageSize': 1,
            'api-key': apiKey,
            'from-date': i,
            'to-date': j,
        }
        res = requests.get(url,params=parameters)
        json_res = res.json()
        json_result = json_res['response']['results']
        for result in json_result:
            links.append(result.get("webUrl"))


In [5]:
fromDateYear = []
toDateYear = []
links = []
generateYears(fromDateYear,toDateYear)
for q in content:
    loopYears(q,links)
    

for l in links:
    print(l)

In [6]:
jsonScrape = []
for l in links:
    page=requests.get(l)
    soup = bs(page.content, 'html.parser'
             )
    title = soup.find("h1",{"class","dcr-125vfar"})
    if title == None:
        continue
    title = title.text
    snippet = soup.find("div",{"class","dcr-1mp5s8u"})
    if snippet == None:
        continue
    author = snippet.text

    if authorSpecified == "":
        snippet = soup.find("summary",{"class","dcr-12fpzem"})
        if snippet is None:
            continue
        date = snippet.text
        snippet = soup.findAll("p",{"class","dcr-xry7m2"})
        fullArticle = ""
        for p in snippet:
            fullArticle += p.text + "\n"
        jsonScrape.append((title,author,date,fullArticle))
    else:
        if authorSpecified == author:
            snippet = soup.find("summary",{"class","dcr-12fpzem"})
            if snippet is None:
                continue
            date = snippet.text
            snippet = soup.findAll("p",{"class","dcr-xry7m2"})
            fullArticle = ""
            for p in snippet:
                fullArticle += p.text + "\n"
            jsonScrape.append((title,author,date,fullArticle))

In [7]:
jsonList = pd.DataFrame(jsonScrape)
jsonList.columns = ['Title', 'Author', 'Date', 'FullArticle']
jsonList

,Title,Author,Date,FullArticle
0,Detroit suspends water shutoffs over Covid-19 ...,Nina Lakhani,Thu 12 Mar 2020 09.00 GMT,Running water will be temporarily restored to ...
1,Self-isolating Tory MP criticises parliament's...,Kate Proctor Political correspondent,Thu 12 Mar 2020 15.31 GMT,The MP Andrew Bridgen has criticised parliamen...
2,Complete shutdown of UK universities due to Co...,Anna Fazackerley,Wed 11 Mar 2020 12.21 GMT,University vice-chancellors have told the gove...
3,Research finds huge impact of interventions on...,Ian Sample Science editor,Wed 11 Mar 2020 18.53 GMT,The rapid spread of coronavirus around the wor...
4,Ireland school closures reveal stark contrast ...,Rory Carroll in Dublin,Thu 12 Mar 2020 15.09 GMT,"Ireland is shutting schools, universities and ..."
5,"US briefing: Biden wins big again, US Covid-19...",Tim Walker,Wed 11 Mar 2020 12.13 GMT,"Good morning, I’m Tim Walker with today’s esse..."
6,Wales to ease Covid lockdown restrictions from...,Damien Gayle and agency,Fri 12 Mar 2021 14.56 GMT,Some semblance of normal life will resume in W...
7,Australia commits $100m to Covid vaccine deal ...,Graham Readfearn and Daniel Hurst,Fri 12 Mar 2021 23.31 GMT,A historic first meeting of leaders of the Qua...
8,First Thing: boost for Biden as Covid aid bill...,Molly Blackall,Thu 11 Mar 2021 10.57 GMT,Good morning\nCongress passed a landmark $1.9t...
9,Manchester schools to launch study into childr...,Paul MacInnes,Thu 11 Mar 2021 22.30 GMT,An unprecedented study into the wellbeing of B...


In [9]:
jsonList.to_json('celestino_dayon_API.json')

In [11]:
scrapeAPI = pd.read_json('celestino_dayon_API.json')

In [12]:
scrapeAPI.head()

,Title,Author,Date,FullArticle
0,Detroit suspends water shutoffs over Covid-19 ...,Nina Lakhani,Thu 12 Mar 2020 09.00 GMT,Running water will be temporarily restored to ...
1,Self-isolating Tory MP criticises parliament's...,Kate Proctor Political correspondent,Thu 12 Mar 2020 15.31 GMT,The MP Andrew Bridgen has criticised parliamen...
2,Complete shutdown of UK universities due to Co...,Anna Fazackerley,Wed 11 Mar 2020 12.21 GMT,University vice-chancellors have told the gove...
3,Research finds huge impact of interventions on...,Ian Sample Science editor,Wed 11 Mar 2020 18.53 GMT,The rapid spread of coronavirus around the wor...
4,Ireland school closures reveal stark contrast ...,Rory Carroll in Dublin,Thu 12 Mar 2020 15.09 GMT,"Ireland is shutting schools, universities and ..."
